In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_new = pd.read_pickle(r'C:\Users\nishi\Desktop\EECS 731 - Data Science\Project\df_new.pkl')

In [3]:
df_new.dtypes

lpep_pickup_datetime     datetime64[ns]
lpep_dropoff_datetime    datetime64[ns]
RatecodeID                        int64
PULocationID                      int64
DOLocationID                      int64
passenger_count                   int64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
payment_type                      int64
trip_type                       float64
trip_duration                   float64
month                             int64
day                               int64
total_custom                    float64
weekend_Trip                      int32
hour                              int64
airport                           int32
dtype: object

In [4]:
# Creating a dataframe for modelling. Models cannot parse datetime columns, so have to drop them

df = df_new[['month','day','hour','PULocationID','DOLocationID','fare_amount','airport']]

In [5]:
df.head()

,month,day,hour,PULocationID,DOLocationID,fare_amount,airport
0,12,6,23,17,61,5.5,0
1,12,6,23,61,49,6.5,0
2,12,6,23,228,188,15.5,0
3,12,6,23,228,26,5.0,0
4,12,6,23,225,225,4.0,0


# Splitting the data into Training and Testing

In [6]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
features = df.drop(['fare_amount'], axis = 1)
labels = df[['fare_amount']]

In [7]:
# Split the data into training and testing sets

train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [8]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (1687283, 6)
Training Labels Shape: (1687283, 1)
Testing Features Shape: (562428, 6)
Testing Labels Shape: (562428, 1)


# Using Random Forest for prediction

In [9]:
%%time
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor()

# Train the model on training data
rf.fit(train_features, train_labels.values.ravel())

Wall time: 1min 43s


In [10]:
print('Accuracy of Random Forest on training set: {:.2f}'
     .format(rf.score(train_features, train_labels)))
print('Accuracy of Random Forest on test set: {:.2f}'
     .format(rf.score(test_features, test_labels)))

Accuracy of Random Forest on training set: 0.90
Accuracy of Random Forest on test set: 0.71


In [11]:
test_features.head()

,month,day,hour,PULocationID,DOLocationID,airport
1974340,2,1,22,74,74,0
884379,7,3,22,41,42,0
2030732,2,1,0,226,129,0
1990037,2,6,16,244,166,0
1866681,3,2,12,52,71,0


In [12]:
test_labels.head()

,fare_amount
1974340,5.5
884379,6.0
2030732,7.0
1990037,11.5
1866681,23.5


# Predicting from Input

In [13]:
check = pd.read_csv(r'C:\Users\nishi\Desktop\EECS 731 - Data Science\Project\test_data.csv')

In [14]:
check.head()

,month,day,hour,PULocationID,DOLocationID,airport
0,2,1,22,74,74,0
1,7,3,22,41,42,0
2,2,1,0,226,129,0
3,2,6,16,244,166,0
4,3,2,12,52,71,0


In [15]:
pred1 = rf.predict(check)

In [16]:
print(pred1)

[ 5.46666667  6.16467047 10.65       12.365      21.65      ]


In [17]:
result = test_features.head()

In [18]:
result['actual_fare'] = test_labels[['fare_amount']].head()
result['predicted_fare'] = pred1

In [19]:
result.head()

,month,day,hour,PULocationID,DOLocationID,airport,actual_fare,predicted_fare
1974340,2,1,22,74,74,0,5.5,5.466667
884379,7,3,22,41,42,0,6.0,6.164670
2030732,2,1,0,226,129,0,7.0,10.650000
1990037,2,6,16,244,166,0,11.5,12.365000
1866681,3,2,12,52,71,0,23.5,21.650000
